In [64]:
#IMPORTANDO AS BIBLIOTECAS
import pandas as pd
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import urlopen
import sqlite3
import os

In [65]:
#Criando as listas para armazenar os dados
ml_url_base = []
ml_urls = []
ml_price = []
ml_seller = []
ml_installment = []
ml_catalog_id = []
ml_catalog_db = []
ml_idetifaction = []
ml_title = []
lista_identification_url = []

In [66]:
#Função para criar os links de busca
def getting_n_creating_mercadolivre_urls(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "-")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "-" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://lista.mercadolivre.com.br/" + df['Urls'] + "_ITEM*CONDITION_2230284_NoIndex_True?#applied_value_name%3DNovo%"

    return df

In [67]:
Df_Raw = getting_n_creating_mercadolivre_urls('Wacom')
Df_Raw

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom-ctl472,https://lista.mercadolivre.com.br/wacom-ctl472...
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom-ctl4100,https://lista.mercadolivre.com.br/wacom-ctl410...
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom-ctl4100wle0,https://lista.mercadolivre.com.br/wacom-ctl410...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom-ctl4100wlk0,https://lista.mercadolivre.com.br/wacom-ctl410...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom-ctl6100wlk0,https://lista.mercadolivre.com.br/wacom-ctl610...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom-dtc133w0a,https://lista.mercadolivre.com.br/wacom-dtc133...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom-pth460,https://lista.mercadolivre.com.br/wacom-pth460...
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom-dtk1660k0a,https://lista.mercadolivre.com.br/wacom-dtk166...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom-pth660,https://lista.mercadolivre.com.br/wacom-pth660...
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom-ctl6100wle0,https://lista.mercadolivre.com.br/wacom-ctl610...


In [68]:
def search_links(url,df):
    # Colocando URL commo Global
    global ml_urls

    # Colocando um tempo mínimo de 3 segundos
    time.sleep(3)

    # Fazendo o response
    response = urlopen(url)
    html = response.read()

    # Criando o BeautifulSoup
    BS = BeautifulSoup(html, 'html.parser')

    # Pegando todos os links da página
    for link in BS.find_all("a", href=True):
        ml_urls.append(link['href'])

    # Pegando as próximas páginas
    try:
        next_page_link = BS.find_all(class_='andes-pagination__arrow-title')[-1].text

        # Vendo se na seta está escrito 'Seguinte'
        if next_page_link == 'Seguinte':
            next_url = BS.find_all(class_='andes-pagination__link ui-search-link')[-1]['href']

            # Realizando o loop da função com o link da próxima página
            search_links(next_url)
    except:
        pass

    # Limpando as urls
    ml_urls = [s for s in ml_urls if 'tracking_id' in s]
    ml_urls = [s for s in ml_urls if 'produto' in s]
    ml_urls = [s for s in ml_urls if df['Brand'][0] in s]

    # Tirando as duplicadas
    ml_urls = list(dict.fromkeys(ml_urls))

In [69]:
for url in tqdm(Df_Raw['Urls_search']):
    search_links(url, Df_Raw)

100%|██████████| 10/10 [00:39<00:00,  3.98s/it]


In [70]:
def search_attributes(url):
    #Criando tempo
    time.sleep(2)

    #Fazendo o response jh
    response = urlopen(url)
    html = response.read()

    #Criando o soup
    BS = BeautifulSoup(html, 'html.parser')

    #Buscando o preço
    try:
        price = BS.find(class_='andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact').text
        ml_price.append(price)
    except:
        ml_price.append("ERRO")

    #Buscando o installment
    try:
        installment = BS.find(class_='ui-pdp-color--GREEN ui-pdp-size--MEDIUM ui-pdp-family--REGULAR').text
        ml_installment.append(installment)
    except:
        installment = BS.find(class_='ui-pdp-color--BLACK ui-pdp-size--MEDIUM ui-pdp-family--REGULAR').text
        ml_installment.append(installment)

    #Catalog
    try:
        catalog = BS.find(class_='ui-pdp-other-sellers mt-24')
        ml_catalog_db.append(catalog)
    except:
        ml_catalog_id.append("NORMAL")

    #Título
    try:
        title = BS.find(class_='ui-pdp-title')
        ml_title.append(title.text)
    except:
        ml_title.append("Erro")

    #Vendedor
    try:
        seller_link = BS.find(class_='ui-pdp-media__action ui-box-component__action')['href']
    except:
        seller_link = "Erro"

    try:
        #Entrando na página do vendedor
        response = urlopen(seller_link)
        html = response.read()

        #Criando o soup
        BS = BeautifulSoup(html, 'html.parser')

        #Achando o nome do seller
        seller_name = BS.find(class_='store-info__name').text

        #Append do nome do seller
        ml_seller.append(seller_name)
    except:
        ml_seller.append(seller_link)

In [71]:
for url in tqdm(ml_urls):
    search_attributes(url)

100%|██████████| 152/152 [10:53<00:00,  4.30s/it]


In [76]:
def create_dataframe(url,seller,price,installment):
    Dataset = pd.DataFrame()

    Dataset['URL'] = url
    Dataset['DATE'] = pd.to_datetime('today', errors='ignore').date()
    Dataset['PRODUCT'] = ml_title

    Dataset['MARKETPLACE'] = 'MERCADO LIVRE'
    Dataset['SELLER'] = seller

    Dataset['Price'] = price
    Dataset['PRICE_1'] = Dataset['Price'].str.partition("reales")[0]
    Dataset["PRICE_2"] = Dataset["Price"].str.partition(" con")[2].str.partition("centavos")[0].str.partition(" ")[2].str.partition(" centavo")[0]

    Dataset['PRICE'] = Dataset['PRICE_1'] + "," + Dataset['PRICE_2']
    Dataset['PRICE'] = Dataset['PRICE'].str.replace(" ","")

    Dataset['Installment'] = installment

    # Arrumando a coluna de installment
    Dataset['PARCEL'] = Dataset['Installment'].str.partition("x")[0]
    Dataset['PARCEL'] = Dataset['PARCEL'].str.extract("(\d+)").astype(int)
    Dataset["reais"] = Dataset["Installment"].str.partition("reales")[0].str.partition("x")[2]
    Dataset["moedas"] = Dataset["Installment"].str.partition(" con")[2].str.partition("centavos")[0].str.partition(" ")[2].str.partition(" centavo")[0]
    # Dataset['Installment'] = Dataset['reais'] + "." + Dataset['moedas']

    Dataset['INSTALLMENT'] = Dataset['reais'] + "," + Dataset['moedas']
    Dataset['INSTALLMENT'] = Dataset['INSTALLMENT'].str.replace(" ","")

    #Dataset['Catalog'] = catalogo

    Dataset['ASIN'] = Dataset['URL'].str.partition("/")[2].str.partition("/")[2].str.partition("/")[2].str.partition("-")[0]
    Dataset['ASIN_2'] = Dataset['URL'].str.partition("/")[2].str.partition("/")[2].str.partition("/")[2].str.partition("-")[2].str.partition("-")[0]

    Dataset['ASIN_CORRETO'] = Dataset['ASIN'] + Dataset['ASIN_2']

    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/HUAWEIOFICIAL?brandId=3562",'HUAWEI LOJA OFICIAL')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MPCEL+MOBILE?brandId=3430",'Mpcel Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MPCEL+MOBILE?brandId=3562",'MPCEL MOBILE')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/ONOFREELETROSERRA?brandId=2156",'Onofre Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/ZOOM_STORE?brandId=3598",'Zoom Store Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/FASTSHOP+OFICIAL?brandId=942",'Fast Shop Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MERCADOLIVRE+ELETRONICOS?brandId=2707",'Mercado Livre Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MERCADOLIVRE+ELETRONICOS?brandId=2707", 'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST?brandId=866", 'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST-STORE+PREMIUM?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST-STORE+SP?brandId=866", 'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/SUNTEKSTOREBR?brandId=2203",'Suntek Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/SUNTEKSTOREBR?brandId=2203",'Suntek Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/OLIST?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/OLIST-STORE+PREMIUM?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/OLIST-STORE+SP?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/E-SPOT?brandId=4166",'Wacom Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/IBYTE.?brandId=3979",'Ibyte Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/INPOWER+INFO?brandId=1655",'INPOWER Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/PICHAUINFORMATICA?brandId=1436",'Pichau Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/PRIMETEK+COMPUTADORES?brandId=2255",'Primetek Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/VLSTORE+INFORMATICA?brandId=3629", 'Leva Digital Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/MIMI2231343?brandId=3396", 'Miranda Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MIMI2231343?brandId=3396",'Miranda Loja Oficial')

    Dataset.drop(['ASIN','ASIN_2'], axis=1, inplace=True)

    Dataset = Dataset[['DATE','URL','MARKETPLACE','SELLER','PRICE','PARCEL','INSTALLMENT','ASIN_CORRETO','PRODUCT']]

    return Dataset

In [78]:
dataset_mercadolivre = create_dataframe(ml_urls,ml_seller,ml_price,ml_installment)

In [79]:
dataset_mercadolivre

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ASIN_CORRETO,PRODUCT
0,2022-06-01,https://produto.mercadolivre.com.br/MLB-206323...,MERCADO LIVRE,Wacom Loja Oficial,"332,90",12,"32,27",MLB2063239068,"Caneta Wacom Intuos Pen Lp-190k Para Ctl472, C..."
1,2022-06-01,https://produto.mercadolivre.com.br/MLB-214042...,MERCADO LIVRE,Wacom Loja Oficial,"379,90",12,"36,83",MLB2140423845,Mesa Digitalizadora One By Wacom Pequena Ctl472 F
2,2022-06-01,https://produto.mercadolivre.com.br/MLB-214046...,MERCADO LIVRE,Wacom Loja Oficial,"399,90",10,"39,99",MLB2140460461,Mesa Digitalizadora One By Wacom Pequena Ctl472 F
3,2022-06-01,https://produto.mercadolivre.com.br/MLB-189829...,MERCADO LIVRE,Wacom Loja Oficial,"399,90",10,"39,99",MLB1898299503,Mesa Digitalizadora One By Wacom Pequena Ctl472
4,2022-06-01,https://produto.mercadolivre.com.br/MLB-189829...,MERCADO LIVRE,Wacom Loja Oficial,"379,90",12,"36,83",MLB1898299487,Mesa Digitalizadora One By Wacom Pequena Ctl472
...,...,...,...,...,...,...,...,...,...
147,2022-06-01,https://produto.mercadolivre.com.br/MLB-214045...,MERCADO LIVRE,Wacom Loja Oficial,"1899,",12,"158,25",MLB2140454573,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...
148,2022-06-01,https://produto.mercadolivre.com.br/MLB-189829...,MERCADO LIVRE,Wacom Loja Oficial,"1909,90",12,"159,16",MLB1898299598,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...
149,2022-06-01,https://produto.mercadolivre.com.br/MLB-192608...,MERCADO LIVRE,TECNOLOGIA OPERA,"1717,",12,"166,46",MLB1926082897,Mesa Digitalizadora Wacom Ctl6100wle0 (c/ Blue...
150,2022-06-01,https://produto.mercadolivre.com.br/MLB-195069...,MERCADO LIVRE,W-TECH-IMPORTS,"1897,",12,"183,91",MLB1950693367,Mesa Digitalizadora Wacom Intuos Ctl6100wle0 B...


In [80]:
dataset_mercadolivre.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Wacom\Files\MercadoL_Wacom.xlsx", index=False)